# Coursework 3: Bilinear Inverse Problems and Low-Rank Matrix Recovery

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 3.1 Test Data Generation (10%)

We consider the low-rank matrix completion problem given by 
$$
    \bm{y} = \mathcal{P}_{\Omega}(\bm{X}) 
$$
where $\bm{X} \in \mathbb{R}^{m \times n}$ is a low rank matrix of rank $r$. 

Data generation: Write $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$, where $\bm{U} \in \mathbb{R}^{m \times r}$, $\bm{G} \in \mathbb{R}^{r \times r}$, and $\bm{V} \in \mathbb{R}^{n \times r}$ are matrices with i.i.d. $\mathcal{N}(0,1)$ Gaussian entries. (Note that by $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$ we are not talking about SVD.)

Design and implement a function `LRMC_data_gen` to generate test data. Provide necessary documentation.

In [2]:
using LinearAlgebra

function gen_Gaussian_Mat(m::Int,n::Int)

    A = randn(m,n)
    for i = 1:n
        v = A[:,i]
        A[:,i] = v/norm(v,2)
    end
    return A
end

function generate_Omega(m::Int, n::Int, size_Omega::Int)
    Omega = zeros(Int,m,n)

    count = 0 
    while count != size_Omega 
        i = rand(1:m); j = rand(1:n)
        if Omega[i,j] != 1
            Omega[i,j] = 1
            count +=1 
        end
    end
    return Omega
end

function linear_projection(Omega::Matrix{Int64},X)
    y = []
    for i = 1:size(X,1) 
        for j = 1:size(X,2)
            if Omega[i,j] == 1 
                append!(y,X[i,j])
            end
        end        
    end
    return y
end


function LRMC_data_gen(r::Int,m::Int,n::Int, Omega::Matrix{Int64}) 
    G = Diagonal(randn(r)) 
    U = gen_Gaussian_Mat(m,r)
    V = gen_Gaussian_Mat(n,r)
    X = U*G*transpose(V)
    y = linear_projection(Omega,X)
    return y,X 
end
r = 3
m = 32; n = 64; size_Omega = Int(1/8 * m * n) ;

Omega = generate_Omega(m,n,size_Omega)

y, X = LRMC_data_gen(r,m,n,Omega)
@show Omega
@show size(y), size_Omega

Omega = [0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1; 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 0 0 

((256,), 256)

## 3.2 Matrix Completion Techniques

In the following, the suggested simulation setup is that $m = 32$, $n=48$, $r$ varies in $2:2:8$, and $|\Omega|/mn$ varies in $\{1/8,~ 1/6,~ 1/4,~ 1/2\}$. 

### 3.2.1 Alternating Minimization (20%)

Design, implement, and run tests for the alternating minimization method for low-rank matrix completion. Use the function name `LRMCRec_AM`. Provide necessary documentation.

In [3]:
function find_v(y, U)
    V = inv(Transpose(U)*U)*Transpose(U)*y
    return V
end

    
    
function Altmin(U, T)
    Ut = U
    for i = 1 : T
        Vt = find_v(y,Ut) #fixed U to find/update V
        Ut = find_v(y,Vt) #fixed V to find/update U
    end
    return Ut,Vt
end

function Altmincomplete(y,U,V,T)
    U_0, S_0, Vt_0 = svd(y;full = true)
    U_t, V_t = Altmin(U_0, T)
    X = U_t*Transpose(V_t)
    return X
end

Altmincomplete (generic function with 1 method)

### 3.2.2 Iterative Hard Thresholding (IHT) (20%)

Design, implement, and run simple tests for the IHT algorithm for low-rank matrix completion. Use the function name `LRMCRec_IHT`. Provide necessary documentation. 

### 3.2.3 Iterative Shrinkage-Thresholding Algorithm (ISTA) (25%)

Design, implement, and run simple tests for ISTA (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ISTA`. Provide necessary documentation. Use simulations to discuss the choice of parameters.

### 3.2.4 Lasso-ADMM (25%)

Design, implement, and run simple tests for an ADMM algorithm (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ADMM`. Provide necessary documentation. Compare ADMM and ISTA in terms of convergence.

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.